# Threading downloads

In [13]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from queue import Queue
from threading import Thread, Lock
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from tqdm import tqdm
import pandas as pd
import os
from typing import Union

In [14]:
def download_single_url(url):
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        parsed_text = ' '.join(soup.text.replace('\n', " ").strip().split())
        return parsed_text
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return None

def download_raw_texts_from_urls(current_urls, max_workers=10):
    raw_texts = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(download_single_url, url): url for url in current_urls}
        
        for future in tqdm(as_completed(futures), total=len(current_urls)):
            result = future.result()
            if result is not None:
                raw_texts.append(result)
    
    return raw_texts

def save_text_to(data: Union[str, list[str]], file_path: str):
    with open(file_path, 'w') as file:
        if type(data) == str:
            file.write(data)
        elif type(data) == list:
            for line in data:
                file.write(line + "\n")
                file.write("<SEP>" + "\n")

In [17]:
directory_path = "/data/kai/forecasting/raw_urls/amd_text.csv"
df = pd.read_csv(directory_path)
df = df[~df["url"].str.contains("zacks.com", na=False)]

data_dir = "/data/kai/forecasting/summary"
ticker = df.iloc[0]["ticker"]

for idx, date_str in tqdm(enumerate(df["timestamp"].unique()), total=len(df["timestamp"].unique())):
    raw_path = f"{data_dir}/{ticker}"
    os.makedirs(raw_path, exist_ok=True)

    current_urls = df[df["timestamp"] == date_str]["url"]
    raw_texts = download_raw_texts_from_urls(current_urls)

    save_text_to(raw_texts, raw_path + f'/{date_str}_raw.txt')

  0%|          | 0/736 [00:00<?, ?it/s]

100%|██████████| 736/736 [16:05<00:00,  1.31s/it]


# vLLM online

In [ ]:
# first spin up the vLLM server. takes a while

# export CUDA_VISIBLE_DEVICES='0,1'
# python -m vllm.entrypoints.openai.api_server --model meta-llama/Meta-Llama-3-70B-Instruct --tensor-parallel-size=2 --disable-log-requests

In [ ]:
from openai import OpenAI
import threading
import queue

In [ ]:
# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

def llm_chat(messages, model="meta-llama/Meta-Llama-3-70B-Instruct"):
    chat_response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return chat_response.choices[0].message.content

def load_text_from(file_path):
    with open(file_path, 'r') as file:
        data = file.read().splitlines()
    return data

In [ ]:
ticker = "aapl"

raw_data = load_text_from("/data/kai/forecasting/summary/aapl_2022-08-19_raw.txt")
raw_data = [d for d in raw_data if d != "<SEP>" and d != ""]

In [ ]:
SUMMARY_PROMPT = f"""
You are a helpful assistant that filters and summarizes stock news specifically for the company with ticker symbol {ticker}. 

Your task is to:
1. Filter out irrelevant information.
2. Provide a concise summary that includes key numbers, growth trends, and the overall market outlook.
3. Mention major stock movements, significant economic indicators, and any notable company-specific news.
4. Avoid making up any information.

If there is no relevant information, the website is blocked, or there is an error message, return nothing.
"""

IGNORE_PROMPT = f"""
Return "<NONE>" if there is an error in the summarization or if the information is irrelevant. Otherwise, return "<TRUE>".
"""

COMBINE_PROMPT = f"""
Combine the summaries into one by following the guidelines:

1. Provide a concise summary that includes key numbers, growth trends, and the overall market outlook.
2. Mention major stock movements, significant economic indicators, and any notable company-specific news.
"""

FINAL_PROMPT = f"""
Given the following stock summaries related to the company with ticker symbol {ticker},

1. Combine them into one comprehensive summary
2. Provide key numbers, growth trends, and the overall market outlook.
3. Mention major stock movements, significant economic indicators, and any notable company-specific news.
4. Avoid making up any information.
"""

def message_template(prompt, content):
    messages=[{
        "role": "system",
        "content": prompt,
    }, {
        "role": "user",
        "content": content
    }]
    return messages

def call_llm_chat(prompt, messages, thread_id, result_queue):
    try:
        response = llm_chat(message_template(prompt, messages))
        result_queue.put((thread_id, response))
    except Exception as e:
        result_queue.put((thread_id, str(e)))

In [ ]:
def batch_call_llm_chat(prompt, data):
    result_queue = queue.Queue()
    threads = []
    for thread_id, content in enumerate(data):
        thread = threading.Thread(target=call_llm_chat, args=(prompt, content, thread_id, result_queue))
        threads.append(thread)
        thread.start()

    # Wait for all threads to complete
    for thread in threads:
        thread.join()

    responses = [None] * len(data)
    while not result_queue.empty():
        thread_id, response = result_queue.get()
        responses[thread_id] = response

    return responses

In [ ]:
# summarize and ignore error messages
summaries = batch_call_llm_chat(SUMMARY_PROMPT, raw_data)
result = batch_call_llm_chat(IGNORE_PROMPT, summaries)

valid_idxs = [i for i, r in enumerate(result) if r != "<NONE>"]
valid_summaries = [summaries[i] for i in valid_idxs]

In [ ]:
valid_summaries

In [ ]:
from transformers import AutoTokenizer

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

token_length = len(tokenizer.encode('\n'.join(valid_summaries)))
max_token_length = 4096

if token_length > max_token_length:
    print("splitting summaries...")
    # Determine how many summaries to combine per chunk
    avg_token_per_summary = token_length / len(valid_summaries)
    summaries_per_chunk = int(max_token_length / avg_token_per_summary)

    # Split the summaries into chunks
    valid_summaries_combined = [
        valid_summaries[i: i + summaries_per_chunk]
        for i in range(0, len(valid_summaries), summaries_per_chunk)
    ]

    combined_summary = batch_call_llm_chat(COMBINE_PROMPT, valid_summaries_combined)
    valid_summaries_combined.append(combined_summary)
else:
    valid_summaries_combined = valid_summaries

final_summary = llm_chat(message_template(FINAL_PROMPT, '\n'.join(valid_summaries_combined)))

In [ ]:
print(final_summary)